In [8]:
import robin_stocks.robinhood as r
import time
import pyotp
import os
from datetime import datetime, timedelta
import config
import pandas as pd
import yfinance as yf
from dateutil.rrule import rrule, WEEKLY
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas_market_calendars as mcal
import pytz

pd.set_option('display.max_rows', None)

In [2]:
def login():
  # Log in to Robinhood using credentials from credentials.py
  totp  = pyotp.TOTP(config.mfa).now()
  time_logged_in = 60*60*24*10
  r.authentication.login(username=config.username,
                          password=config.password,
                          expiresIn=time_logged_in,
                          scope='internal',
                          mfa_code=totp,
                          store_session=True)
  print("LOGGED INTO ROBINHOOD SUCCESSFULLY at", datetime.now())

def logout():
    r.authentication.logout()
    print("LOGGED OUT OF ROBINHOOD SUCCESSFULLY at", datetime.now())

login()
#logout()

LOGGED INTO ROBINHOOD SUCCESSFULLY at 2023-08-09 15:45:49.392983


In [3]:
#fucntion to send the stats via email

def send_email(subject, message):
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    
    # Your credentials
    MY_ADDRESS = 'gavaserviceacc@gmail.com'
    PASSWORD = 'jmsbuenkgfxigstq'
    
    s.login(MY_ADDRESS, PASSWORD)

    # create a message
    msg = MIMEMultipart()
    
    msg['From']=MY_ADDRESS
    msg['To']="sarrafgsarraf@gmail.com"
    msg['Subject']=subject
    
    msg.attach(MIMEText(message, 'plain'))
    
    s.send_message(msg)
    
    s.quit()

In [4]:
#fucntion used in calculating days return
def get_prev_day_closing_price(symbol):
    """
    Returns the closing price of the stock from the previous market day.
    """
    # Get the NYSE calendar
    nyse = mcal.get_calendar('NYSE')

    # Get today's date, delayed by 3 hours to ensure that the market data for today is available
    today = datetime.now(pytz.timezone('America/New_York')) - timedelta(hours=3)

    # Create market schedule for the past 10 days
    market_schedule = nyse.schedule(start_date=today - timedelta(days=10), end_date=today)

    # Check if market is closed today
    if today > market_schedule.iloc[-1]['market_close']:
        previous_market_day = market_schedule.iloc[-1].name.date()
    else:
        previous_market_day = market_schedule.iloc[-2].name.date()

    # Convert it to a string for comparison
    previous_market_day_str = previous_market_day.strftime('%Y-%m-%d')

    # Get the historical prices for the past 10 days
    historicals = r.stocks.get_stock_historicals(symbol, interval='day', span='week', bounds='regular')

    # Find the entry for the previous day and return the closing price
    for entry in historicals:
        if entry['begins_at'][:10] == previous_market_day_str:
            return float(entry['close_price'])

    # If no data is found for the previous day, raise an exception
    raise Exception(f"No historical data found for {symbol} on {previous_market_day_str}")

get_prev_day_closing_price("NVDA")

446.64

In [4]:
#fucntion to fetch portfolio stats

holdings_stats = []
tickers = []
def stats():
  msg = ""
  # Get portfolio info
  holdings = r.build_holdings()
  user_profile = r.build_user_profile()

  all_holdings_value = sum(float(value['equity']) for value in holdings.values())
  cash_balance = int(float(user_profile['cash']))
  portfolio_value = all_holdings_value + cash_balance

  total_return = 0.0
  total_today_return = 0.0
  for key, value in holdings.items():
    tickers.append(key)
    average_cost = float(value['average_buy_price'])
    current_price = float(value['price'])
    quantity = float(value['quantity'])
    equity = float(value['equity'])
    profit_loss = (current_price - average_cost) * quantity

    # Get the closing price from the previous day
    prev_day_closing_price = get_prev_day_closing_price(key)  # Replace this with the correct API call
    today_return = (current_price - prev_day_closing_price) * quantity

    total_return += profit_loss
    total_today_return += today_return
    holdings_stats.append({
        'Symbol': key,
        'Name': value['name'],
        'Equity $': equity,
        'Quantity': quantity,
        'Current Price $': current_price,
        'Allocation in %': (equity / portfolio_value) * 100,
        'Total Return $': profit_loss,
        'Todays Return $': today_return
    })

  # Get individual stock performance
  print("\nIndividual Stock Allocation and Profit/Loss as of", datetime.now())
  holdings_stats_df = pd.DataFrame(holdings_stats)
  display(holdings_stats_df)
  #msg += "\n" + holdings_stats_df.to_string(index=False)

  print("\nPortfolio Value: ${:.2f}".format(portfolio_value))
  print("All Holdings Value: ${:.2f}".format(all_holdings_value))
  print("Cash Balance: ${}".format(cash_balance))
  print("Warning: The actual cash values might not be accurate because of unsettled orders. Verify using RobinHood online portal")

  profit_loss_symbol = "+" if total_return >= 0 else "-"
  print("\nTotal Profit/Loss: {} ${:.2f}  as of {}".format(profit_loss_symbol, abs(total_return), datetime.now()))
  #msg += "\nTotal Profit/Loss: {}${:.2f}".format(profit_loss_symbol, abs(total_return))

  today_return_symbol = "+" if total_today_return >= 0 else "-"
  print("\nToday's Return: {} ${:.2f}  as of {}".format(today_return_symbol, abs(total_today_return), datetime.now()))
  #msg += "\nToday's Return: {}${:.2f}".format(today_return_symbol, abs(total_today_return))

  #send_email("Your daily Robinhood Stats", msg)

stats()

In [9]:
#improved version to get stats. Does not set emails

def fetch_stats():
    holdings_stats = []
    tickers = []
    try:
        # Get portfolio info
        holdings = r.build_holdings()
        user_profile = r.build_user_profile()

        all_holdings_value = sum(float(value['equity']) for value in holdings.values())
        cash_balance = int(float(user_profile['cash']))
        portfolio_value = all_holdings_value + cash_balance

        total_return = 0.0
        total_today_return = 0.0
        for key, value in holdings.items():
            tickers.append(key)
            average_cost = float(value['average_buy_price'])
            current_price = float(value['price'])
            quantity = float(value['quantity'])
            equity = float(value['equity'])
            profit_loss = (current_price - average_cost) * quantity

            # Get the closing price from the previous day
            prev_day_closing_price = get_prev_day_closing_price(key)  # Replace this with the correct API call
            today_return = (current_price - prev_day_closing_price) * quantity

            total_return += profit_loss
            total_today_return += today_return
            holdings_stats.append({
                'Symbol': key,
                'Name': value['name'],
                'Equity $': equity,
                'Quantity': quantity,
                'Current Price $': round(current_price,2),
                'Allocation in %': round((equity / portfolio_value) * 100,2),
                'Total Return $': round(profit_loss,2),
                'Todays Return $': round(today_return,2)
            })
        return holdings_stats, portfolio_value, all_holdings_value, cash_balance, total_return, total_today_return

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def print_stats(holdings_stats, portfolio_value, all_holdings_value, cash_balance, total_return, total_today_return):
    # Get individual stock performance
    print("\nIndividual Stock Allocation and Profit/Loss as of", datetime.now())
    holdings_stats_df = pd.DataFrame(holdings_stats)
    
    display(holdings_stats_df)

    print("\nPortfolio Value: ${:.2f}".format(portfolio_value))
    print("All Holdings Value: ${:.2f}".format(all_holdings_value))
    print("Cash Balance: ${}".format(cash_balance))
    print("Warning: The actual cash values might not be accurate because of unsettled orders. Verify using RobinHood online portal")

    profit_loss_symbol = "+" if total_return >= 0 else "-"
    print("Total Profit/Loss: {} ${:.2f}  as of {}".format(profit_loss_symbol, abs(total_return), datetime.now()))

    today_return_symbol = "+" if total_today_return >= 0 else "-"
    print("Today's Return: {} ${:.2f}  as of {}".format(today_return_symbol, abs(total_today_return), datetime.now()))

stats = fetch_stats()
if stats is not None:
    print_stats(*stats)



Individual Stock Allocation and Profit/Loss as of 2023-08-09 16:04:02.305132


,Symbol,Name,Equity $,Quantity,Current Price $,Allocation in %,Total Return $,Todays Return $
0,NVDA,NVIDIA,1231.05,2.886749,426.45,11.03,577.44,-58.28
1,MSFT,Microsoft,811.62,2.516250,322.55,7.27,37.15,-8.81
2,HON,Honeywell International,466.35,2.464326,189.24,4.18,-17.42,3.92
3,TSLA,Tesla,1020.71,4.216752,242.06,9.14,58.60,-32.22
4,AAPL,Apple,509.52,2.857970,178.28,4.56,28.96,-4.34
5,SPY,SPDR S&P 500 ETF,1028.39,2.304925,446.17,9.21,70.24,-5.95
6,VTWG,Vanguard Russell 2000 Growth Index Fund,636.58,3.630326,175.35,5.70,32.90,-7.30
7,XSVM,Invesco S&P SmallCap Value with Momentum ETF,653.85,13.006810,50.27,5.86,46.68,-1.30
8,QQQ,Invesco QQQ,741.89,2.015013,368.18,6.65,90.59,-7.70
9,GOOGL,Alphabet Class A,51.85,0.399886,129.66,0.46,14.96,-0.70



Portfolio Value: $11163.09
All Holdings Value: $11017.09
Cash Balance: $146
Total Profit/Loss: + $1064.12  as of 2023-08-09 16:04:02.403352
Today's Return: - $141.66  as of 2023-08-09 16:04:02.403415


In [5]:
def get_all_open_orders():
    """
    This function fetches and prints all open orders.
    """
    try:
        # Get all open orders
        open_orders = []
        open_orders = r.orders.get_all_open_stock_orders()
        # Check if there are any open orders
        if not open_orders:
            print("No open orders.")
            return []

        order_data = []

        for order in open_orders:
            instrument_name = r.stocks.get_name_by_url(order['instrument'])

            order_data.append({
                'ID': order['id'],
                'Instrument': instrument_name,
                'Quantity': order['quantity'],
                'Type': order['type'],
                'Side': order['side'],
                'State': order['state'],
            })

        #df = pd.DataFrame(order_data)
        #display(df)
        return order_data

    except Exception as e:
        print(f"An error occurred: {e}")

get_all_open_orders()


Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
No open orders.


[]

In [6]:
def execute_transactions(tickers_and_amounts, action):
    """
    This function takes in a list of ticker-amount pairs and an action (either 'buy' or 'sell') and executes the 
    transactions using the Robinhood API.
    """
    for ticker, amount in tickers_and_amounts:
        try:
            if action == 'buy':
                r.orders.order_buy_fractional_by_price(ticker, amount)
                print(f"Successfully BUY order placed for ${amount} of {ticker}")
            elif action == 'sell':
                r.orders.order_sell_fractional_by_price(ticker, amount)
                print(f"Successfully SELL order placed for ${amount} of {ticker}")
            else:
                print(f"Invalid action: {action}. Please specify either 'buy' or 'sell'")
        except Exception as e:
            print(f"An error occurred while trying to {action} {ticker}: {e}")
        time.sleep(15)  # pause for 15 seconds
        print("waiting for 15 seconds")

execute_transactions([('VTWG', 10), ('AAPL', 10), ('CAT', 2) ,('MSFT', 20),('IYF', 10),('META', 5),('XSVM', 10),('XLE', 10),('IYF', 10),('AMZN', 5),('SCHB', 2),('LUMN', 2),('QQQ', 10),('SPY', 20),('NVDA', 10),('HON', 10),('TSLA', 30),('JNJ', 10)], 'buy')  # for buying
#execute_transactions([('AAPL', 1), ('TSLA', 1)], 'sell')  # for selling

Successfully BUY order placed for $10 of VTWG
waiting for 15 seconds
Successfully BUY order placed for $10 of AAPL
waiting for 15 seconds
Successfully BUY order placed for $2 of CAT
waiting for 15 seconds
Successfully BUY order placed for $20 of MSFT
waiting for 15 seconds
Successfully BUY order placed for $10 of IYF
waiting for 15 seconds
Successfully BUY order placed for $5 of META
waiting for 15 seconds
Successfully BUY order placed for $10 of XSVM
waiting for 15 seconds
Successfully BUY order placed for $10 of XLE
waiting for 15 seconds
Successfully BUY order placed for $10 of IYF
waiting for 15 seconds
Successfully BUY order placed for $5 of AMZN
waiting for 15 seconds
Successfully BUY order placed for $2 of SCHB
waiting for 15 seconds
Successfully BUY order placed for $2 of LUMN
waiting for 15 seconds
Successfully BUY order placed for $10 of QQQ
waiting for 15 seconds
Successfully BUY order placed for $20 of SPY
waiting for 15 seconds
Successfully BUY order placed for $10 of NVDA

In [6]:
def cancel_orders():
    """
    This function fetches all open orders and then cancels them.
    """
    try:
        # Get all open orders
        
        open_orders = get_all_open_orders()

        if not open_orders:  # If the open_orders list is empty
            print("No open orders to cancel.")
            return

        # Cancel each order
        for order in open_orders:
            order_id = order['ID']
            cancel_response = r.orders.cancel_stock_order(order_id)

            # Check if the cancellation was successful
            if 'detail' in cancel_response:
                print(f"Failed to cancel order {order_id}: {cancel_response['detail']}")
            else:
                print(f"Successfully canceled order {order_id}")

    except Exception as e:
        print(f"An error occurred: {e}")
    open_orders = get_all_open_orders()
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(open_orders)
    # Display the DataFrame
    display(df)

cancel_orders()


Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
No open orders.
No open orders to cancel.


In [ ]:
# the code below takes the start and end date and downloads the stock prices of past 1 month and saves it into a csv file
# list of tickers needs to be provided or it will take the already available global list of tickers. 
# Uses the Yahoo Finance unofficial API 

def fetch_data_one_month(start_date, end_date, tickers):
    # Generate list of Mondays and Saturdays between the start and end dates
    mondays = list(rrule(WEEKLY, dtstart=start_date, until=end_date, byweekday=0))  # 0 = Monday
    saturdays = list(rrule(WEEKLY, dtstart=start_date, until=end_date, byweekday=5))  # 5 = Saturday
    week_ranges = list(zip(mondays, saturdays))

    folder_name = f"stock_prices_{start_date.strftime('%Y-%m-%d')}_to_{end_date.strftime('%Y-%m-%d')}"
    filename = folder_name+"/"+folder_name+".csv"
    os.makedirs(folder_name, exist_ok=True)

    # Fetch data for each week and append to CSV
    for start, end in week_ranges:
        data = yf.download(tickers, start=start.strftime('%Y-%m-%d'), end=end.strftime('%Y-%m-%d'), interval="1m") # Fetch data
        data = data.swaplevel(axis=1).sort_index(axis=1)  # Swap and sort column levels 
        data = data.round(3)
        data.to_csv(filename, mode='a')  # Append data to CSV
        time.sleep(10)  # Sleep for 10 seconds between each request to respect rate limits

    # Create a text file with all the ticker names and start and end dates
    with open(f"{folder_name}/info.txt", "w") as f:
        f.write(f"Tickers: {', '.join(tickers)}\n")
        f.write(f"Number of tickers: {len(tickers)}\n")
        f.write(f"Start date: {start_date.strftime('%Y-%m-%d')}\n")
        f.write(f"End date: {end_date.strftime('%Y-%m-%d')}\n")
        
    print("Downloaded data for all tickers for dates mentioned. CSV and TXT files created")

# Use the function
start_date = datetime(2023, 6, 11) #should be a date four week earlier from today (Sunday prefered)
end_date = datetime(2023, 7, 8) #should be a date today (Saturday Prefered)
fetch_data_one_month(start_date, end_date, tickers)
